In [166]:
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
from sklearn.model_selection import train_test_split

torch.manual_seed(43)

In [167]:
transform = transforms.Compose([transforms.ToTensor()])
datasets = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
x_train, x_test, y_train, y_test = train_test_split(datasets.data, datasets.targets, test_size=0.2, random_state=42)
train_loader = DataLoader(dataset=list(zip(x_train, y_train)), batch_size=32)
test_loader = DataLoader(dataset=list(zip(x_test, y_test)), batch_size=32)

In [168]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.fc1 = nn.Linear(784,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,10)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


In [169]:
import torch.optim as optim

student = Student().to("cuda")
optimizer = optim.SGD(student.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [170]:
for epoch in range(10):
    running_loss = 0.0

    for data in train_loader:
        inputs, truth = data
        optimizer.zero_grad()
        pred = student(inputs.to("cuda").float())
        truth = truth.to("cuda")
        loss = criterion(pred, truth)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss:{running_loss}")

Epoch 1, Loss:1574934.330837965
Epoch 2, Loss:3450.7629561424255
Epoch 3, Loss:3450.53914809227
Epoch 4, Loss:3450.4870343208313
Epoch 5, Loss:3450.4713122844696
Epoch 6, Loss:3450.4508938789368
Epoch 7, Loss:3450.403995037079
Epoch 8, Loss:3450.4160418510437
Epoch 9, Loss:3450.406290769577
Epoch 10, Loss:3450.380619764328


In [171]:
misclassifications = 0
student.eval()
with torch.no_grad():
    for data in test_loader:
            inputs, truth = data
            optimizer.zero_grad()
            pred = student(inputs.to("cuda").float())
            pred =torch.argmax(pred, dim=1).float()
            truth = truth.to("cuda").float()
            misclassifications += torch.sum(pred != truth).item()

In [173]:
misclassifications

10684

In [175]:
len(test_loader)*32

12000

In [172]:
misclassifications / (len(test_loader)*32)

0.8903333333333333

# Almost 0.89% misclassification